In [ ]:
!pip install pymongo_ssh
!pip install bertopic
!pip install --upgrade numba

# Hate specific Topic model
We want to build a topic model that is trained only on messages that were classified as offensive by at least one of the binary classifiers. With this we can get an overview over the topics in the hateful Telegram messages.

In [1]:
import pymongo
from pymongo_ssh import MongoSession
import re
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy
from random import shuffle
german_stop_words = stopwords.words('german')

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Again in order to connect to the MongoDB we need the private key which is located on my Google Drive, so we mount it in order to access the key.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
session = MongoSession(
    host='138.246.233.159',
    port=22,
    user='ubuntu',
    key='/content/drive/My Drive/NLP/id_hsd',
    uri='mongodb://127.0.0.1:27017'
)

client = session.connection['telegram']

print(client.collection_names())

#session.stop()

['channels', 'errors', 'entities', 'users', 'messages']


We do the same preprocessing steps as for the other topic model

In [4]:
def append_link_preview(msg):
  text = msg['text']
  if msg['link_preview_title'] is not None:
    text += "\n" + msg['link_preview_title']
  if msg['link_preview'] is not None:
    text += "\n" + msg['link_preview']
  return text

def replace_URL(string):
    return re.sub('http[s]?://\S+', '', string)

def replace_username(string):
    return re.sub('@\S+', '<USERNAME>', string)

def replace_URL_and_username(string):
    return replace_username(replace_URL(string))

ow get all the messages that were classified as offensive by at least one of the binary classifiers.

In [ ]:
messages = client.messages
query = {'$or': [{'hate_hasoc_1920.label': 'HOF'}, {'hate_germeval_1819.label': 'OFFENSE'}, {'hate_germeval_18.label': 'OFFENSE'}, {'hate_germeval_19.label': 'OFFENSE'}, 
                 {'hate_covid_2021.label': 'abusive'}, {'hate_hasoc_2020.label': 'HOF'}]}
query_res = messages.find(query, {"_id": 0, "text": 1, "link_preview_title": 1, "link_preview": 1})
message_list = []
for msg in query_res:
    if msg['text'] is not None:
        text = append_link_preview(msg)
        preprocessed_msg = replace_URL_and_username(text)
        message_list.append(preprocessed_msg)
print(len(message_list))

1335370


We randomly pick 100.000 of those messages, as this should suffice to get an overview over the relevant topics.

In [ ]:
shuffle(message_list)
message_list = message_list[:100000]
print(len(message_list))

100000


In [ ]:
vectorizer_model = CountVectorizer(ngram_range=(1, 1), stop_words=german_stop_words, min_df=5)
model = BERTopic(language = "german", nr_topics=30, calculate_probabilities = False, vectorizer_model=vectorizer_model, verbose=True)
topics, probabilities = model.fit_transform(message_list)

2021-07-26 11:01:26,905 - BERTopic - Transformed documents to Embeddings
2021-07-26 11:02:39,108 - BERTopic - Reduced dimensionality with UMAP
2021-07-26 11:02:48,468 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-07-26 11:04:41,060 - BERTopic - Reduced number of topics from 1353 to 31


In [ ]:
print(model.get_topic_info())

    Topic  Count                                               Name
0      -1  48938                   -1_menschen_deutschland_wer_gibt
1       0   6095                            0_töten_menschen_tv_wer
2       1   3682            1_frauen_jeffrey_ghislaine_vergewaltigt
3       2   2494                  2_schweden_menschen_comedian_geht
4       3   2186                       3_jesus_herr_kirche_christus
5       4   2158                     4_maske_masken_video_wikileaks
6       5   2038                 5_grünen_klima_twitter_klimawandel
7       6   1737                   6_rassismus_nazis_nazi_deutschen
8       7   1728                 7_15_österreich_frankreich_italien
9       8   1716               8_virus_coronavirus_infizierte_viren
10      9   1620                      9_politiker_partei_trump_wahl
11     10   1610                  10_flynn_mafia_militär_bundeswehr
12     11   1486                           11_medien_report_22_2020
13     12   1476  12_migranten_flüchtlinge_deuts

In [ ]:
from pprint import pprint
for topic in range(30):
  if model.get_topic(topic):
    pprint(model.get_topic(topic))

[('töten', 0.009230086995506849),
 ('menschen', 0.008018424599633414),
 ('tv', 0.006851487611784234),
 ('wer', 0.0067570937172192185),
 ('video', 0.006571483343132763),
 ('deutschland', 0.006563307245331919),
 ('bitte', 0.006416698238914258),
 ('danke', 0.006387993673125308),
 ('10', 0.006125343867145579),
 ('news', 0.0060800781677246745)]
[('frauen', 0.013557174332363617),
 ('jeffrey', 0.013138294379376995),
 ('ghislaine', 0.01283703245527329),
 ('vergewaltigt', 0.009335199949330584),
 ('omnia', 0.00891809690412216),
 ('ungefiltert', 0.008829675950621094),
 ('veritas', 0.008736047713169173),
 ('unzensiert', 0.0087155359618994),
 ('menschen', 0.007631741252736557),
 ('gibt', 0.006736460742984918)]
[('schweden', 0.011802193336578068),
 ('menschen', 0.01121699043568975),
 ('comedian', 0.008495314337829645),
 ('geht', 0.008028140823563446),
 ('deutschland', 0.0078121689078576115),
 ('lachen', 0.007518371079459918),
 ('who', 0.00728592871740802),
 ('püppchen', 0.007094219180337766),
 ('vid

In [ ]:
model.save("/content/drive/My Drive/NLP/model_hate")

In [ ]:
model = BERTopic.load('/content/drive/My Drive/NLP/model_hate')